## Byte-at-a-time ECB decryption (Simple)
 Copy your oracle function to a new function that encrypts buffers under ECB mode using a _consistent_ but _unknown_ key (for instance, assign a single random key, once, to a global variable).

Now take that same function and have it append to the plaintext, BEFORE ENCRYPTING, the following string: 

```
Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK
```

|Spoiler alert.|
| ------------ |
|Do not decode this string now. Don't do it.|

Base64 decode the string before appending it. _Do not base64 decode the string by hand; make your code do it._ The point is that you don't know its contents.

What you have now is a function that produces: 

`AES-128-ECB(your-string || unknown-string, random-key)`

It turns out: you can decrypt "unknown-string" with repeated calls to the oracle function!

Here's roughly how: 

1. Feed identical bytes of your-string to the function 1 at a time --- start with 1 byte ("A"), then "AA", then "AAA" and so on. Discover the block size of the cipher. You know it, but do this step anyway.
2. Detect that the function is using ECB. You already know, but do this step anyways.
2. Knowing the block size, craft an input block that is exactly 1 byte short (for instance, if the block size is 8 bytes, make "AAAAAAA"). Think about what the oracle function is going to put in that last byte position.
2. Make a dictionary of every possible last byte by feeding different strings to the oracle; for instance, "AAAAAAAA", "AAAAAAAB", "AAAAAAAC", remembering the first block of each invocation.
2. Match the output of the one-byte-short input to one of the entries in your dictionary. You've now discovered the first byte of unknown-string.
2. Repeat for the next byte.


|Congratulations.|
| -------------- |
|This is the first challenge we've given you whose solution will break real crypto. Lots of people know that when you encrypt something in ECB mode, you can see penguins through it. Not so many of them can decrypt the contents of those ciphertexts, and now you can. If our experience is any guideline, this attack will get you code execution in security tests about once a year.|


In [1]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from os import urandom
from base64 import b64decode
import string

In [2]:
KEY = urandom(16) # 128-bit AES key

In [3]:
FLAG = b64decode("Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK")

In [4]:
def encryption_oracle(plaintext:bytes):
    plaintext = plaintext + FLAG
    padded = pad(plaintext, AES.block_size)
    cipher = AES.new(KEY, AES.MODE_ECB)
    return cipher.encrypt(padded)


### Confirm block size
Passing a byte string of length $1$ to the oracle function returns a ciphertext of length $144$.

Consider $144 = len(FLAG) + 1 + 16$ where $1$ is the length of the input and $16$ is the expected length of padding if the unknown string length is $127$.

In [5]:
ciphertext = encryption_oracle(b'0')
len(ciphertext)

144

Show that ciphertext length increases by the block size $16$ when passing a byte string of length $6$, which means that the unknown string length is $144 - 6 = 138$ due to expected padding behavior when the plaintext to be padded has a length that is a multiple of block size.

In [11]:
for n in range(1, AES.block_size):
    ciphertext = encryption_oracle(b'0' * n)
    if len(ciphertext) > 144:
        print(n, len(ciphertext))
        break

6 160


### Detect ECB

In [6]:
# Returns (is_ecb:boolean, counts:dict) tuple, given ciphertext.
def detect_ecb(ciphertext):
    assert len(ciphertext) % AES.block_size == 0
    counts = {}
    for i in range(0, len(ciphertext), AES.block_size):
        block = ciphertext[i:i + AES.block_size]
        if block not in counts:
            counts[block] = 0
        counts[block] += 1
    return max(counts.values()) > 1, counts

In [7]:
NUM_BLOCKS = 3
input = b'\x00' * AES.block_size * NUM_BLOCKS

In [11]:
ciphertext = encryption_oracle(input)
is_ecb, _ = detect_ecb(ciphertext)
print(is_ecb)

True


### Discover first byte of unknown string

In [8]:
data = dict()
for ch in string.ascii_letters:
    b = bytes(ch, encoding='utf-8')
    input = b'\x00' * 15 + b
    ciphertext = encryption_oracle(input)
    first_block = ciphertext[:AES.block_size]
    assert len(first_block) == AES.block_size
    data[first_block] = b


In [9]:
len(data)

52

In [10]:
ciphertext = encryption_oracle(b'\x00' * 15)

In [11]:
data[ciphertext[:AES.block_size]]

b'R'

### Try to generalize approach

In [5]:
# Returns dict of encrypted block to candidate for next unknown char, given known chars so far.
def make_dictionary(known):
    d = dict()
    for ch in string.printable:
        input = None
        if len(known) < AES.block_size: # Still working first block.
            input = known + ch # Append candidate char.
            input = input.zfill(AES.block_size) # Left fill string with zeros.
        else:
            input = known[-15:] + ch # LOOK!!
        input = bytes(input, encoding='utf-8') # Need bytes for oracle function.

        ciphertext = encryption_oracle(input)

        first_block = ciphertext[:AES.block_size]
        assert len(first_block) == AES.block_size # Sanity check.
        d[first_block] = ch

    return d


In [6]:
# Returns (start, stop) tuple to index into the ciphertext slice for the next block.
def block_indices(known):
    block_num = len(known) // AES.block_size # Floor division.
    start = block_num * AES.block_size
    stop = (block_num + 1) * AES.block_size
    return start, stop

In [16]:
known = ''
for _ in range(0, 138): # 138 is the calculated flag length.
    d = make_dictionary(known)
    count = AES.block_size - (len(known) % AES.block_size + 1)
    ciphertext = encryption_oracle(b'0' * count)

    start, stop = block_indices(known)
    #print(count, start, stop) # Uncomment to better understand what happens at block boundaries.
    known += d[ciphertext[start:stop]]

print(known, len(known))


Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by
 138
